## Rocchio & Retrieval

Given the lucene index with term vectors stored, the following retriever with Rocchio PRF can be run.

In this version 6 (v6) notebook, I aim to run a search for the max MAP for TFIDF term weights and report corresponding params.

In [1]:
topicFilePath = './trec6.xml'  # 50 queries

In [2]:
import xml.etree.ElementTree as ET

tree = ET.parse(topicFilePath)
topics = tree.getroot()

In [3]:
import lucene
from org.apache.lucene.search import IndexSearcher
from org.apache.lucene.index import DirectoryReader
from org.apache.lucene.store import FSDirectory
from org.apache.lucene.queryparser.classic import QueryParser
from org.apache.lucene.search.similarities import BM25Similarity
from org.apache.lucene.search.similarities import LMJelinekMercerSimilarity
from org.apache.lucene.search.similarities import LMDirichletSimilarity
from org.apache.lucene.analysis.en import EnglishAnalyzer
from java.io import File

from org.apache.lucene.search import BooleanQuery
from org.apache.lucene.search import BooleanClause
from org.apache.lucene.search import TermQuery
from org.apache.lucene.search import BoostQuery
from org.apache.lucene.index import Term

In [4]:
# run this again if VM is not initialized already
lucene.initVM()

In [5]:
index_path = './index/'
directory = FSDirectory.open(File(index_path).toPath())
indexReader = DirectoryReader.open(directory)

### Rocchio

In [28]:
FIELDNAME = 'CONTENT'       # Lucene index field name

import math

# calculating avgdl for queries. Used in BM25_query().
analyzer = EnglishAnalyzer()
query_lens = []
for topic in topics:
    queryKeywordsField = 'title'     # other fields are 'desc'and 'narr'
    q = topic.find(queryKeywordsField).text.strip()
    escaped_q = QueryParser(FIELDNAME, analyzer).escape(q)      # a few titles had '/' in them which 
                                                                # EnglishAnalyzer was not able to parse
                                                                # without escaping those special characters
    query = QueryParser(FIELDNAME, analyzer).parse(escaped_q)
    query_terms = [term.strip()[len(FIELDNAME)+1:] for term in query.toString().split()]
    query_lens.append(len(query_terms))
avgdl_query = sum(query_lens)/len(query_lens)

# calculating avgdl for the corpus. Used in BM25_docVec().
N = indexReader.numDocs()
avgdl_collection = indexReader.getSumTotalTermFreq(FIELDNAME)/N


def tf_idf_query(term, query_terms):
    # returns TF-IDF weight for the given term in query
    D = len(query_terms)
    N = indexReader.numDocs()
    tf = query_terms.count(term)
    df = indexReader.docFreq(Term(FIELDNAME, term))
    weight = (tf/D)*(math.log(N/(df+1)))
    return weight


def tf_idf_docVec(docVec, D):
    # tf-idf weight calculation for all the terms in the document vector
    N = indexReader.numDocs()       # no. of total docs in the corpus
    for t in docVec:
        tf = docVec[t][0]
        df = docVec[t][1]
        idf = math.log(N/(df+1))
        docVec[t] = (tf/D)*idf
    
    return docVec


def BM25_query(term, query_terms, k1=0.8, b=0.4):
    # returns Okapi BM25 weight for the given term in query
    D = len(query_terms)
    N = indexReader.numDocs()
    tf = query_terms.count(term)
    df = indexReader.docFreq(Term(FIELDNAME, term))
    idf = math.log(1+((N-df+0.5)/(df+0.5)))
    weight = ((tf*(1+k1))/(tf+k1*((1-b)+(b*D/avgdl_query))))*idf
    return weight


def BM25_docVec(docVec, D, k1=0.8, b=0.4):
    # Okapi BM25 weight calculation for all the terms in the document vector
    N = indexReader.numDocs()       # no. of total docs in the corpus
    for t in docVec:
        tf = docVec[t][0]
        df = docVec[t][1]
        idf = math.log(1+((N-df+0.5)/(df+0.5)))
        docVec[t] = ((tf*(1+k1))/(tf+k1*((1-b)+(b*D/avgdl_collection))))*idf
    
    return docVec


def getDocumentVector(luceneDocid, weightScheme):
    # returns document vector in dictionary form with tf-idf weights
    from org.apache.lucene.util import BytesRefIterator
    
    docVec = {}                     # doc vector, which will have terms as keys and 
                                    # its tf-idf weight in the doc as values
    
    D = 0                           # doc length, i.e., total no. of tokens in the doc
    terms = indexReader.getTermVector(luceneDocid, FIELDNAME)
    iterator = terms.iterator()
    for term in BytesRefIterator.cast_(iterator):
        t = term.utf8ToString()
        tf = iterator.totalTermFreq()    # termFreq of term,t
        df = indexReader.docFreq(Term(FIELDNAME, t))    # docFreq of term,t
        D += tf
        docVec[t] = [tf,df]
        
    if weightScheme == 'TFIDF':
        docVec = tf_idf_docVec(docVec, D)
    elif weightScheme == 'BM25':
        docVec = BM25_docVec(docVec, D)
    
    return docVec


def rocchio_PRF(query, top_k_docs, N, alpha, beta, weightScheme):
    """Implements Rocchio's relevance feedback and returns a modified query

    Args:
        query (org.apache.lucene.search.Query): lucene parsed version of the initial/original query
        top_k_docs (lucene._lucene.JArray_object): scoreDocs returned after performing search with top k results
        N (int): number of terms to be in the returned modified query
        alpha (float): weight for original query
        beta (float): weight for positive feedback
        weightScheme (string): TFIDF or BM25 for term weighting

    Returns:
        list: expanded/modified query list of string query terms
    """
    
    # processing JQuery object to extract query terms in form of a list
    query_terms = [term.strip()[len(FIELDNAME)+1:] for term in query.toString().split()]
    
    # creating query vector Q0
    Q0_vector = {}
    for term in query_terms:
        if weightScheme == 'TFIDF':
            Q0_vector[term] = tf_idf_query(term, query_terms)
        elif weightScheme == 'BM25':
            Q0_vector[term] = BM25_query(term, query_terms)
    
    sumRelDocsVector = {}     # Rel for Relevant, NRel for Non-relevant
    numRel = 0
    for scoreDoc in top_k_docs:
        docVec = getDocumentVector(scoreDoc.doc, weightScheme)
        numRel += 1
        # vector addition of sumRelDocsVector and docVec
        for term in docVec:
            if term in sumRelDocsVector:
                sumRelDocsVector[term] += docVec[term]
            else:
                sumRelDocsVector[term] = docVec[term]
    
    
    r = {term: sumRelDocsVector[term]/numRel for term in sumRelDocsVector}    # normlaized Relevant Docs Vector
    
    # final Rocchio formula for Qm 
    expanded_query = [[term, alpha*Q0_vector.get(term,0) + beta*r.get(term,0)] for term in set(Q0_vector) | set(r)]
    
    expanded_query.sort(key = lambda x: x[1], reverse=True)   # sorted (descending) the expanded query list as per term scores
    Qm_with_scores = expanded_query[:int(N)]     # selecting top N expanded query terms
    
    # weighting expanded query terms
    booleanQuery = BooleanQuery.Builder()
    for item in Qm_with_scores:
        t = Term(FIELDNAME, item[0])
        tq = TermQuery(t)
        boostedTermQuery = BoostQuery(tq, item[1])
        BooleanQuery.setMaxClauseCount(4096)
        booleanQuery.add(boostedTermQuery, BooleanClause.Occur.SHOULD)
    modifiedQuery = booleanQuery.build()
    
    return modifiedQuery   # modified query

### LMJM + Rocchio Retrieval

In [29]:
def lmjm_rocchio(numPRD, N, alpha, beta, weightScheme='TFIDF'):
    """ Performs LMJM search with Rocchio pseudo relevance feedback 
        on a set of queries and output the result in a file

    Args:
        numPRD: no. of pseudo relevant docs
        N: no. of expansion terms
        alpha, beta: Rocchio model parameters
        weightScheme (string): TFIDF or BM25 for term weighting
        
    Returns:
        None
    """
     
    
    model = 'lmjm'
    LAMBDA = 0.4   # LM-JM baseline lambda parameter
    similarityModel = LMJelinekMercerSimilarity(LAMBDA)

    # change result file path below
    if weightScheme == 'BM25' or weightScheme == 'TFIDF':
        rocchioOutputPath = f"./Rocchio_output/{weightScheme}/LMJM_Rocchio_numPRD={numPRD}_N={N}_alpha={alpha}_beta={beta}_{weightScheme}.res"
    else:
        print('Warning: weightScheme entered not a valid parameter value. Taking default weightScheme: TFIDF')
        weightScheme = 'TFIDF'
        rocchioOutputPath = f"./Rocchio_output/{weightScheme}/LMJM_Rocchio_numPRD={numPRD}_N={N}_alpha={alpha}_beta={beta}_{weightScheme}.res"
    
    f = open(rocchioOutputPath, 'w')

    # setting up the searcher
    analyzer = EnglishAnalyzer()    # used same analyzer as indexer
    index_path = './index/'
    directory = FSDirectory.open(File(index_path).toPath())
    searcher = IndexSearcher(DirectoryReader.open(directory))
    # setting the similarity model
    searcher.setSimilarity(similarityModel)

    print('\nRetrieving ...')

    # search on 50 queries from the topic file 'trec6.xml'
    for topic in topics:
        qidField = 'num'
        queryKeywordsField = 'title'     # other fields are 'desc'and 'narr'

        qid = topic.find(qidField).text.strip()
        q = topic.find(queryKeywordsField).text.strip()

        escaped_q = QueryParser(FIELDNAME, analyzer).escape(q)      # a few titles had '/' in them which 
                                                                    # EnglishAnalyzer was not able to parse
                                                                    # without escaping those special characters
        query = QueryParser(FIELDNAME, analyzer).parse(escaped_q)

        print(f'Rocchio {weightScheme}, numPRD = {numPRD}, N = {N}, alpha = {alpha}, beta = {beta}; qid = {qid}, retrieving & writing ...', end=' ')

        # getting the top pseudo relevant docs using the searcher
        scoreDocs = searcher.search(query, numPRD).scoreDocs

        # Rocchio expanded query retrieval
        modified_query = rocchio_PRF(query, scoreDocs, N=N, alpha=alpha, beta=beta, weightScheme=weightScheme)

        # getting the top k search results using the searcher
        k = 1000
        scoreDocs = searcher.search(modified_query, k).scoreDocs

        # writing all k doc results in a .res file in TREC format
        rank = 0
        results = ''
        for scoreDoc in scoreDocs:
            rank += 1
            doc = searcher.doc(scoreDoc.doc)
            # f.write(f"{qid}\tQ0\t{doc.get('DOCID')}\t{rank}\t{scoreDoc.score}\taman_lmjm_{LAMBDA}-rocchio_{alpha}_{beta}\n")
            results += f"{qid}\tQ0\t{doc.get('DOCID')}\t{rank}\t{scoreDoc.score}\taman_lmjm_{LAMBDA}-rocchio_{alpha}_{beta}\n"
        
        f.write(results)

        print('complete!')

    f.close()
    print('Search completed! Search results exported to a .res file in the current directory.\n')

### Finding max MAP for LMJM+Rocchio-TFIDF

In [ ]:
# numPRD = 35
# N = 100
# alpha = 1
# beta = 30

# lmjm_rocchio(numPRD=numPRD,N=N,alpha=alpha,beta=beta, weightScheme='BM25')
# lmjm_rocchio(numPRD=numPRD,N=N,alpha=alpha,beta=beta, weightScheme='TFIDF')

In [30]:
import os
import subprocess

def getBetaForTopMAPs(beta_list, top):
    MAPs = []
    
    for beta in beta_list:
        rocchioOutputPath = f"./Rocchio_output/{weightScheme}/LMJM_Rocchio_numPRD={numPRD}_N={N}_alpha={alpha}_beta={beta}_{weightScheme}.res"
        result = subprocess.run(['trec_eval', 'trec678_robust.qrel', rocchioOutputPath], stdout=subprocess.PIPE)
        shell_output_string = result.stdout.decode('utf-8')
        if shell_output_string.split()[15] == 'map':
            map_value = float(shell_output_string.split()[17])
            MAPs.append([numPRD,N,alpha,beta,map_value])
        else:
            raise Exception('map_value index misalignment')
    
    MAPs.sort(key = lambda x: x[4], reverse=True)
    
    top_betas = []
    top_MAPs = set()
    counter = 0
    for item in MAPs:
        top_betas.append(item[3])
        if item[4] not in top_MAPs:
            counter += 1
            top_MAPs.add(item[4])
        if counter >= top:
            break
        
    return top_betas


def beta_loop():
    beta_step_size = 10
    min_beta_step_size = 2.5
    beta_list = list(range(5,46,beta_step_size))
    for beta in beta_list:
        if f'LMJM_Rocchio_numPRD={numPRD}_N={N}_alpha={alpha}_beta={beta}_{weightScheme}.res' not in os.listdir(f'./Rocchio_output/{weightScheme}/'):
            lmjm_rocchio(numPRD=numPRD,N=N,alpha=alpha,beta=beta,weightScheme=weightScheme)
    beta_step_size /= 2
    while beta_step_size >= min_beta_step_size:
        top_betas = getBetaForTopMAPs(beta_list, top=3)
        beta_list = []
        for beta in top_betas:
            beta_list.append(beta+beta_step_size)
            beta_list.append(beta-beta_step_size)
        beta_list = sorted(list(set(beta_list)))
        for beta in beta_list:
            if f'LMJM_Rocchio_numPRD={numPRD}_N={N}_alpha={alpha}_beta={beta}_{weightScheme}.res' not in os.listdir(f'./Rocchio_output/{weightScheme}/'):
                lmjm_rocchio(numPRD=numPRD,N=N,alpha=alpha,beta=beta,weightScheme=weightScheme)
        beta_step_size /= 2


def getNforTopMAPs(N_list, top):
    maxMAPs = []
    
    for N in N_list:
        MAPs = []
        for rocchioOutputPath in os.listdir(f'./Rocchio_output/{weightScheme}/'):
            if rocchioOutputPath.startswith(f'LMJM_Rocchio_numPRD={numPRD}_N={N}_alpha={alpha}_beta='):
                result = subprocess.run(['trec_eval', 'trec678_robust.qrel', f'./Rocchio_output/{weightScheme}/'+rocchioOutputPath], stdout=subprocess.PIPE)
                shell_output_string = result.stdout.decode('utf-8')
                if shell_output_string.split()[15] == 'map':
                    map_value = float(shell_output_string.split()[17])
                    MAPs.append([numPRD,N,alpha,beta,map_value])
                else:
                    raise Exception('map_value index misalignment')
        maxMAPs.append([N,max(MAPs, key=lambda x:x[4])[4]])
    
    maxMAPs.sort(key = lambda x: x[1], reverse=True)
    
    top_Ns = []
    top_MAPs = set()
    counter = 0
    for item in maxMAPs:
        top_Ns.append(item[0])
        if item[1] not in top_MAPs:
            counter += 1
            top_MAPs.add(item[1])
        if counter >= top:
            break
    
    return top_Ns

In [ ]:
# Algorithm for making lmjm_rocchio runs for all the appropriate params

weightScheme = 'TFIDF'
alpha = 1

numPRD_step_size = 5
for numPRD in range(20,46,numPRD_step_size):
    N_step_size = 10
    N_list = list(range(50,161,N_step_size))
    for N in N_list:
        beta_loop()
    top_Ns = getNforTopMAPs(N_list,top=5)      # func considers max MAP for each N for choosing top Ns
    N_list = []
    N_step_size /= 2
    for N in top_Ns:
        N_list.append(N+N_step_size)
        N_list.append(N-N_step_size)
    N_list = sorted(list(set(N_list)))
    for N in N_list:
        beta_loop()
    

In [ ]:
# Evaluating MAPs for all the 1893 lmjm_rochhio runs

import subprocess
import csv

def paramsFromFilename(filename):
    split_list = filename.split('_')
    numPRD = int(split_list[2].split('=')[1])
    N = int(float(split_list[3].split('=')[1]))
    alpha = float(split_list[4].split('=')[1])
    beta = float(split_list[5].split('=')[1])
    return numPRD,N,alpha,beta

f = open(f'./Rocchio_output/params_vs_MAP_Rocchio_{weightScheme}.tsv', 'w')
tsv_writer = csv.writer(f, delimiter='\t')
tsv_writer.writerow(['method','numPRD','N','alpha','beta','MAP'])

MAPs = []

counter = 0

sorted_filenames  = sorted(os.listdir(f'./Rocchio_output/{weightScheme}/'), key=paramsFromFilename)
for filename in sorted_filenames:
    numPRD,N,alpha,beta = paramsFromFilename(filename)
    rocchioOutputPath = f'./Rocchio_output/{weightScheme}/' + filename
    result = subprocess.run(['trec_eval', 'trec678_robust.qrel', rocchioOutputPath], stdout=subprocess.PIPE)
    shell_output_string = result.stdout.decode('utf-8')
    if shell_output_string.split()[15] == 'map':
        map_value = float(shell_output_string.split()[17])
        MAPs.append([numPRD,N,alpha,beta,map_value])
        tsv_writer.writerow(['lmjm_0.4_rocchio_TFIDF',numPRD, N, alpha, beta, map_value])
        counter += 1
        print(f'{counter} ... done!')
    else:
        raise Exception('map_value index misalignment')
    
f.close()

In [61]:
# highest MAP value and corresponding params

res = max(MAPs, key=lambda x:x[4])
print(res)
print(f'max MAP = {res[4]}, for numPRD = {res[0]}, N = {res[1]}, alpha = {res[2]}, beta = {res[3]}')

[35, 115, 1.0, 20.0, 0.2834]
max MAP = 0.2834, for numPRD = 35, N = 115, alpha = 1.0, beta = 20.0
